In [1]:
import bkutils
import warnings
import os
import pandas as pd
from tqdm.notebook import tqdm

warnings.filterwarnings(action="ignore")
os.getcwd()

seoul_bike = pd.read_parquet(
    "/Users/yangwoolee/git_repo/seoul_bike_dash/parquet/220607_bike_record.parquet.gzip"
)

bkstation_info = pd.read_csv(
    "assets/bkstation_info.csv", encoding="CP949", index_col=0
)
near_sub = pd.read_csv("/Users/yangwoolee/git_repo/data/static/near_sub_station.csv", encoding="CP949")
near_bus = pd.read_csv("assets/near_bus_200m.csv", encoding="CP949", index_col=0)

In [177]:
j = near_sub.dropna(subset='sub_name')['bi_st_id'].tolist()

In [5]:
seoul_bike["weekday"] = seoul_bike["date"].dt.weekday
data = seoul_bike.groupby("weekday").size()
data_numpy = data.to_numpy()

print(data_numpy)

[3780702 3669305 4293641 4122031 4169084 3504779 3458548]


In [17]:
station = pd.read_csv(
    "assets/seoul_bike_station_01_12.csv", encoding="CP949", index_col=0
)

asd = station.reset_index(drop=False)
asd[asd['st_id'] == 207]

,st_id,st_name,district,latitude,longtitude
90,207,여의나루역 1번출구 앞,영등포구,37.526989,126.932098


In [178]:
import ast
# [ast.literal_eval(near_bus['bike_id'][i]) for i in range(0,len(near_bus))]
a = []
near_bus = near_bus.dropna()
for i in near_bus.index :
    try : 
        k = ast.literal_eval(near_bus['bike_id'][i])
        if type(k) != int :
            a.extend(k)
    except :
        print(i)
k = list(set(a))

In [168]:
#1. 전체 0~5시 대여 비율
seoul_bike['date'].dt.hour.value_counts(normalize=True).sort_index()[1:6].sum() # ==> 0.0470
#2. 야간 버스 노선있는 0~5시 대여비율



0.035560997092757306

In [179]:
seoul_bike[seoul_bike["st_id1"].isin(j)]['date'].dt.hour.value_counts(normalize=True).sort_index()[1:6]

1    0.011499
2    0.007639
3    0.005132
4    0.004056
5    0.005120
Name: date, dtype: float64

In [184]:
kk= seoul_bike[seoul_bike["st_id1"].isin(k)]['date'].dt.hour.value_counts(normalize=True).sort_index()[1:6].sum()
kkk= seoul_bike[seoul_bike["st_id1"].isin(k)]['date'].dt.hour.value_counts().sum()
print(kkk)

18403400


In [185]:
bb= seoul_bike[~seoul_bike["st_id1"].isin(k)]['date'].dt.hour.value_counts(normalize=True).sort_index()[1:6].sum()
bbb= seoul_bike[~seoul_bike["st_id1"].isin(k)]['date'].dt.hour.value_counts().sum()



8594690


In [186]:
len(seoul_bike)

26998090

In [7]:
import numpy as np
from typing import List

def haversine_np( coor1 :List[int],coor2 :List[int]
    ) -> np.array:

        lon1=coor1[0]; lat1=coor1[1]
        lon2=coor2[0]; lat2=coor2[1]

        """
        Calculate the great circle distance between two points
        on the earth (specified in decimal degrees)

        All args must be of equal length.

        """

        lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = (
            np.sin(dlat / 2.0) ** 2
            + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
        )

        c = 2 * np.arcsin(np.sqrt(a))
        m = 6367 * c * 1000
        return m

array([ 1660.78304536,  1635.08420908,  1749.29930456, ...,
        7676.53065922, 14192.96131344,  9412.42871467])

In [4]:
%time
station = pd.read_csv(
    "assets/seoul_bike_station_01_12.csv", encoding="CP949", index_col=0
)
bkstation_info = pd.read_csv(
    "assets/bkstation_info.csv", encoding="CP949", index_col=0
)

station[['latitude','longtitude']].T.values

addd = haversine_np([37.534863, 126.90241], station[['latitude','longtitude']].T.values)

station['dist'] = addd

k = station[station['dist'] <= 500].reset_index(drop=False)['st_id']

result = bkstation_info[bkstation_info.value.isin(k.tolist())]




CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


NameError: name 'haversine_np' is not defined

0        102
1        103
2        104
3        105
4        106
        ... 
2627    2714
2628    2200
2629    3304
2630    2292
2631    2543
Name: value, Length: 2632, dtype: int64

,value,label,coor,num
124,244,영등포삼환아파트 앞,"[37.530079, 126.905708]",27058
126,247,당산역 10번출구 앞,"[37.533688, 126.902107]",104822
144,267,삼성화재 사옥 옆,"[37.535961, 126.8983]",30739
145,268,그랜드컨벤션센터 앞,"[37.534718, 126.900002]",34071
148,272,당산육갑문,"[37.535339, 126.903679]",98559
1909,3201,당산skv1센터,"[37.530735, 126.89917]",20404
1910,3202,당산롯데캐슬프레스티지,"[37.52869, 126.903862]",36572


In [2]:
bkstation_info.head()

,value,label,coor,num
0,102,망원역 1번출구 앞,"[37.555649, 126.910629]",62074
1,103,망원역 2번출구 앞,"[37.554951, 126.910835]",51577
2,104,합정역 1번출구 앞,"[37.550629, 126.914986]",25702
3,105,합정역 5번출구 앞,"[37.550007, 126.914825]",20609
4,106,합정역 7번출구 앞,"[37.548645, 126.912827]",63341


In [3]:
import numpy as np


def haversine_np(lon1: float, lat1: float, lon2: float, lat2: float
) -> np.array:
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        np.sin(dlat / 2.0) ** 2
        + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    )

    c = 2 * np.arcsin(np.sqrt(a))
    m = 6367 * c * 1000
    return m

def raw_data( val: int) -> pd.DataFrame:
        quert_st_id1 = seoul_bike[seoul_bike["st_id1"] == val]
        quert_st_id2 = seoul_bike[seoul_bike["st_id2"] == val]
        filtered_data = pd.concat(
            [quert_st_id1, quert_st_id2], axis=0
        ).drop_duplicates()
        filtered_data.drop(columns="index", inplace=True)
        bm = (filtered_data["st_id1"] == val) & (filtered_data["st_id2"] == val)
        filtered_data = filtered_data[~bm]

        # # 반납
        # filtered_data_end = filtered_data[
        #     (filtered_data["st_id2"] == val) & (filtered_data["st_id1"] != val)
        # ]

        return filtered_data

In [7]:
from typing import Dict
# Dict[int,str,list,int,float,float]
def arrStation(st_id : int) :
    # 대여소 추출기록
    filtered_data = raw_data(st_id)
    st_id1_record = filtered_data.groupby(by='st_id1').size()
    st_id2_record = filtered_data.groupby(by='st_id2').size()

    all_record = st_id1_record.add(st_id2_record, fill_value=0)

    # 대여기록 100건 이상 총 200개 대여소만 추출하기
    sort_info = all_record[all_record>50].sort_values(ascending=False)[1:201] 
    station_id = sort_info.index.to_list()
    
    result = bkstation_info.query('value == @station_id').reset_index(drop=True) 
    # return result

    result_station = []
    for j in result['value'] :
        BM = (filtered_data["st_id2"] == j )
        st_id1_time = (
            filtered_data[BM]["riding_time"]
            .value_counts()
            .sort_values(ascending=False)
        )
        BM = filtered_data["st_id1"] == j
        st_id2_time = (
            filtered_data[BM]["riding_time"]
            .value_counts()
            .sort_values(ascending=False)
        )
        ### 1to2 걸린시간 + 2to1 걸린시간 합
        all_rent = st_id1_time.add(st_id2_time, fill_value=0)

        ### 1to2 & 2to1 대여기록 합
        total_record = all_rent.sum()

        k = []
        i = 2
        ### 기록 많은 순만 종합
        while len(k) < 1:
            k = all_rent[all_rent >= (total_record / i)]
            i = i * 1.5

        ### 대여시간
        ind = k.index
        ### 대여기록
        val = k.values
        ### 대여기록 합
        a = k.sum()
        ### 대여시간 * 대여기록
        asddd = sum([a * b for a, b in zip(ind, val)])
        # 평균 시간
        ddddd = asddd / a

        # 올림
        val = round(ddddd, 0)

        ### 평균거리 계산
        BM = (filtered_data["st_id2"] == j )
        st_id1_time = (
            filtered_data[BM]["dist"]
            .value_counts()
            .sort_values(ascending=False)
        )
        BM = filtered_data["st_id1"] == j
        st_id2_time = (
            filtered_data[BM]["dist"]
            .value_counts()
            .sort_values(ascending=False)
        )
        all_rent = st_id1_time.add(st_id2_time, fill_value=0)
        dist = pd.cut(all_rent.index,bins=50).value_counts().sort_values(ascending=False)
        num=3
        vals = [dist[:num].index[a].mid * dist.iloc[a] for a in range(3)]
        avg_dist = sum(vals)/sum(dist.to_list()[:num])


        result_station.append([val,total_record,(round(avg_dist/1000,2))])

    df = pd.DataFrame(result_station)
    finish = pd.concat([result, df],axis=1)
    finish.columns = ['value','label','coor','num','time','record','dist']
    # result['dist'] = avg_dist_list
    minmax = dict(
    mintime = finish['time'].max(),
    maxtimen = finish['time'].min(),
    minrecord = finish['record'].max(),
    maxrecord = finish['record'].min(),
    mindist = finish['dist'].max(),
    maxdist=finish['dist'].min(),
    )
    
    
    return finish,minmax # min과 max를 설정할 수 있도록 순번 잊지말기 

abb = arrStation(754)


In [10]:
abb[0].sort_values(by='record', ascending=False)

,value,label,coor,num,time,record,dist
15,744,신목동역 2번 출구,"[37.543842, 126.882545]",41765,3.0,6066.0,0.71
20,755,목동1단지아파트 상가 앞 (월촌중학교 버스정류소 옆),"[37.537868, 126.881409]",38499,4.0,2380.0,1.03
16,746,목동2단지 상가,"[37.536503, 126.877747]",49344,4.0,1451.0,0.83
14,735,영도초등학교,"[37.53643, 126.871521]",46288,4.0,1204.0,1.25
25,766,신목동역 3번출구,"[37.54426, 126.88325]",35103,4.0,1159.0,0.81
12,726,목동3단지 시내버스정류장,"[37.534389, 126.869598]",47004,6.0,1136.0,1.07
27,769,CBS방송국 앞,"[37.527855, 126.874672]",66738,10.0,993.0,1.73
23,764,목동청소년수련관,"[37.531029, 126.875893]",41361,9.0,895.0,1.46
30,773,"염창역 3, 4번 출구 사이","[37.546707, 126.874573]",27241,8.0,723.0,1.54
37,793,목동5단지아파트 534동,"[37.53355, 126.878143]",24498,8.0,641.0,1.22


In [21]:
import json
import requests
import time
import ast
def route_coor(
        departure_station,
        arrival_station,):
    # ) -> List :
        """departure_station과 arrival_station은 모두 dataframe으로!"""
        if isinstance(departure_station, pd.Series):
            raise TypeError(f'DataFrame 가능... 현재 Type : {type(departure_station)}')

        if isinstance(arrival_station, pd.Series):
            raise TypeError(f'DataFrame 가능... 현재 Type {type(arrival_station)}')
        if departure_station.empty or arrival_station.empty:
            raise ValueError('Value is Empty!')

        departure = ast.literal_eval(departure_station['coor'].values[0])
        arrival = ast.literal_eval(arrival_station['coor'].values[0])

        for i in range(3):
            url = "https://apis.openapi.sk.com/tmap/routes/pedestrian?version=1"

            payload = {
                "angle": 0,
                "speed": 0,
                "reqCoordType": "WGS84GEO",
                "searchOption": "0",
                "resCoordType": "WGS84GEO",
                "sort": "index",
                "startX": departure[1],
                "startY": departure[0],
                "endX": arrival[1],
                "endY": arrival[0],
                "startName": "출발",
                "endName": "도착",
            }
            headers = {
                "Accept": "application/json",
                "Content-Type": "application/json",
                "appKey": "l7xxfdc75c1509a74ecdba02bf5e024ee9d5",
            }

            response = requests.post(url, json=payload, headers=headers).text

            api_data = json.loads(response).get("features")
            if api_data != None:
                time.sleep(0.3)
                break
            print("!!!!!!!!!!!!!", api_data, f"{i+1} 회 시도중")

        coor_raw_data = [
            api_data[i].get("geometry").get("coordinates")
            for i in range(0, len(api_data))
        ]

        finish_data = []
        for data in coor_raw_data:
            # data = api_data[1].get('geometry').get('coordinates')
            if type(data[0]) == list:
                for i in data:
                    finish_data.append(i)
            else:
                pass
        return pd.DataFrame(finish_data)[[1, 0]].values  ## 위도 경도 위치 바꾸기

In [ ]:
for j in abb['value'][:1] :
    ### 평균거리 계산
    BM = (filtered_data["st_id2"] == j )
    st_id1_time = (
        filtered_data[BM]["dist"]
        .value_counts()
        .sort_values(ascending=False)
    )
    BM = filtered_data["st_id1"] == j
    st_id2_time = (
        filtered_data[BM]["dist"]
        .value_counts()
        .sort_values(ascending=False)
    )
    all_rent = st_id1_time.add(st_id2_time, fill_value=0)
    dist = pd.cut(all_rent.index,bins=50).value_counts().sort_values(ascending=False)
    num=3
    vals = [dist[:num].index[a].mid * dist.iloc[a] for a in range(3)]
    avg_dist = sum(vals)/sum(dist.to_list()[:num])
    print(dist)
    print(j,avg_dist)

In [ ]:
abb

In [231]:
result_station = []
for j in abb['value'] :
    BM = (filtered_data["st_id2"] == j )
    st_id1_time = (
        filtered_data[BM]["riding_time"]
        .value_counts()
        .sort_values(ascending=False)
    )
    BM = filtered_data["st_id1"] == j
    st_id2_time = (
        filtered_data[BM]["riding_time"]
        .value_counts()
        .sort_values(ascending=False)
    )
    ### 1to2 걸린시간 + 2to1 걸린시간 합
    all_rent = st_id1_time.add(st_id2_time, fill_value=0)

    ### 1to2 & 2to1 대여기록 합
    total_record = all_rent.sum()

    k = []
    i = 2
    ### 기록 많은 순만 종합
    while len(k) < 1:
        k = all_rent[all_rent >= (total_record / i)]
        i = i * 1.5

    ### 대여시간
    ind = k.index
    ### 대여기록
    val = k.values
    ### 대여기록 합
    a = k.sum()
    ### 대여시간 * 대여기록
    asddd = sum([a * b for a, b in zip(ind, val)])
    # 평균 시간
    ddddd = asddd / a

    # 올림
    result = round(ddddd, 0)

    ### 평균거리 계산
    BM = (filtered_data["st_id2"] == j )
    st_id1_time = (
        filtered_data[BM]["dist"]
        .value_counts()
        .sort_values(ascending=False)
    )
    BM = filtered_data["st_id1"] == j
    st_id2_time = (
        filtered_data[BM]["dist"]
        .value_counts()
        .sort_values(ascending=False)
    )
    all_rent = st_id1_time.add(st_id2_time, fill_value=0)
    dist = pd.cut(all_rent.index,bins=50).value_counts().sort_values(ascending=False)
    num=3
    vals = [dist[:num].index[a].mid * dist.iloc[a] for a in range(3)]
    avg_dist = sum(vals)/sum(dist.to_list()[:num])


    result_station.append([result,total_record,avg_dist])


